In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta


In [ ]:
# 1. 카테고리 설정
# ---------------------------------------------------------
num_rows = 130  

# 구독, 주거/통신은 1번만 등장하도록 함.(아래에서 다룸)
category_map_random = {
    '식비': ['스타벅스', '맥도날드', '김밥천국', '배달의민족', '이마트24', 'GS25', '투썸플레이스', '쿠팡이츠', '구내식당', '삼겹살집'],
    '교통비': ['지하철', '택시', '버스', 'SK주유소', '카카오T', '코레일', 'GS칼텍스'],
    '쇼핑': ['쿠팡', '다이소', '무신사', '올리브영', '네이버쇼핑', '유니클로', '무인양품', 'ZARA'],
    '의료/건강': ['약국', '내과', '헬스장할부', '치과', '정형외과'],
    '문화/여가': ['CGV', '교보문고', '롯데월드', 'PC방', '볼링장', '야구장'],
    '교육': ['토익접수', '교재구입', '인프런', '영어 강의'],
    '기타': ['모임회비', '경조사비', '더치페이']
}

# 주거/통신 (고정비 1)
housing_internet = ['월세', '관리비', 'SKT통신비', '도시가스', '인터넷요금']

# 구독 리스트 (고정비 2 - 매달 1회만 발생)
subscription_items = ['넷플릭스', '유튜브프리미엄', '멜론', '쿠팡와우', 'ChatGPT']

payment_methods = ['신용카드', '체크카드', '계좌이체', '현금']

categories = list(category_map_random.keys())
# 랜덤으로 등장하는 카테고리의 가중치 설정
weights = [0.37, 0.21, 0.17, 0.05, 0.14, 0.04, 0.02] 



In [ ]:

# 2. 로직 함수 정의
# ---------------------------------------------------------
start_date = datetime(2026, 1, 1)

def modified_date(base_date):
    r = random.random()
    rand_day = random.randint(0, 31)
    target_date = base_date + timedelta(days=rand_day)

    if r < 0.8: return target_date.strftime("%Y-%m-%d")
    elif r < 0.9: return target_date.strftime("%m.%d")
    else: return target_date.strftime("%Y/%m/%d")

def modified_amount(category, fixed_value=None):
    """금액 생성"""
    r = random.random()
    
    if fixed_value is not None:
        base_amount = fixed_value
    else:
        if category == '교육': base_amount = random.randint(2, 10) * 10000 
        elif category == '쇼핑': base_amount = random.randint(5, 150) * 1000 
        elif category == '식비': base_amount = random.randint(4, 25) * 1000 
        elif category == '교통비': base_amount = random.randint(15, 250) * 100 
        elif category == '기타': base_amount = random.randint(3, 10) * 10000 
        
        # 구독 금액 설정
        elif category == '구독': base_amount = random.randint(10, 25) * 1000 
            
        else: base_amount = random.randint(5, 50) * 1000
        
    if r < 0.6: return base_amount
    elif r < 0.9: return f"{base_amount:,}"
    else: return str(base_amount)

def determine_essential(category, desc):
    """필수 지출 판별"""
    if category in ['주거/통신', '의료/건강', '교통비']:
        if desc == '택시': return False 
        return True
    
    if category == '식비':
        if desc in ['구내식당', '김밥천국', '이마트24', 'GS25']: return True
        return False
        
    if category == '쇼핑':
        if desc == '다이소': return True 
        return False 
        
    if category == '교육': return True 

    return False



In [ ]:


# 3. 데이터 조립

data = []

# 주거/통신 생성 (1회씩)
for item in housing_internet:
    if item == '월세': price = 700000; fix_date = datetime(2026, 1, 25)
    elif item == '관리비': price = random.randint(10, 15) * 10000; fix_date = None
    elif item == 'SKT통신비': price = random.randint(5, 8) * 10000; fix_date = None
    elif item == '인터넷요금': price = random.randint(2, 3) * 10000; fix_date = None
    else: price = random.randint(1, 3) * 10000; fix_date = None
    
    date_str = fix_date.strftime("%Y-%m-%d") if fix_date else modified_date(start_date)
    amount_val = modified_amount('주거/통신', fixed_value=price)
    
    data.append([date_str, amount_val, '주거/통신', item, '계좌이체', True, True]) 


#  구독 생성 ( 1회씩만 생성)
for item in subscription_items:
    # 구독은 랜덤 날짜 생성 (매달 갱신일이 다르다고 가정하거나, 그냥 랜덤 날짜)
    date_str = modified_date(start_date)
    amount_val = modified_amount('구독') # 금액은 함수에서 1~2.5만원 사이로 나옴
    
    # 구독은 무조건 Fixed=True, Essential=False (줄일 수 있음)
    data.append([date_str, amount_val, '구독', item, '신용카드', True, False])


# 나머지 랜덤 데이터 생성
# (전체 목표 개수) - (주거 개수) - (구독 개수)
remaining_rows = num_rows - len(housing_internet) - len(subscription_items)

for _ in range(remaining_rows):
    cat = random.choices(categories, weights=weights, k=1)[0]
    desc = random.choice(category_map_random[cat])
        
    date_val = modified_date(start_date)
    amount_val = modified_amount(cat)
    pay_method = random.choice(payment_methods)
    
    # 고정비 여부 (교육만 확률적, 나머지는 False)
    is_fixed = random.choice([True, False]) if cat == '교육' else False
    
    # 필수 여부
    is_essential = determine_essential(cat, desc)
    if random.random() < 0.05: is_essential = not is_essential

    data.append([date_val, amount_val, cat, desc, pay_method, is_fixed, is_essential])

# 섞기
random.shuffle(data)

# DataFrame 생성
df = pd.DataFrame(data, columns=['date', 'amount', 'category', 'description', 'payment_method', 'is_fixed', 'essential'])



In [ ]:


# ---------------------------------------------------------
# 4. 결과 확인 및 저장
# ---------------------------------------------------------

print("\n[구독 항목 확인 ]")
print(df[df['category'] == '구독'][['date', 'description', 'amount', 'is_fixed']])

# CSV 저장
filename = "expense_data_essential.csv"
df.to_csv(filename, index=False, encoding='utf-8-sig')
print(f"\n파일 생성 완료")